# Supplementary information for damage mapper tool

Development of the damage mapper tool can be broken down into three parts:

1. A function `damage_zones` to calculate the coordinates of the surface zero location and the airblast damage radii
2. A function to plot the blast zones on a map
3. Functions to locate the postcodes (or postcode sectors) within the blast zones `get_postcodes_by_radius` and look up the population in these postcodes `get_population_of_postcodes`.

For the extension task you will need to develop additional functions.

## Airblast damage

The rapid deposition of energy in the atmosphere is analogous to an explosion and so the environmental consequences of the airburst can be estimated using empirical data from atmospheric explosion experiments [(Glasstone and Dolan, 1977)](https://www.dtra.mil/Portals/61/Documents/NTPR/4-Rad_Exp_Rpts/36_The_Effects_of_Nuclear_Weapons.pdf).

The main cause of damage close to the impact site is a strong (pressure) blastwave in the air, known as the **airblast**. Empirical data suggest that the pressure in this wave $p$ (in Pa) (above ambient, also known as overpressure), as a function of explosion energy $E_k$ (in kilotons of TNT equivalent), burst altitude $z_b$ (in m) and horizontal range $r$ (in m), is given by:

\begin{equation*}
p(r) = 3.14 \times 10^{11} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-1.3} + 1.8 \times 10^{7} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-0.565}
\end{equation*}

For airbursts, we will take the total kinetic energy lost by the asteroid at the burst altitude as the burst energy $E_k$. For cratering events, we will define $E_k$ as the **larger** of the total kinetic energy lost by the asteroid at the burst altitude or the residual kinetic energy of the asteroid when it hits the ground.

Note that the burst altitude $z_b$ is the vertical distance from the ground to the point of the airburst and the range $r$ is the (great circle) distance along the surface from the "surface zero point," which is the point on the surface that is closest to the point of the airburst (i.e., directly below).

The following threshold pressures can then be used to define different degrees of damage.

|  Damage Level |  Description    | Pressure (kPa) |
|:-------------:|:---------------:|:--------------:|
|  1  |  ~10% glass windows shatter    |     1.0      |
|  2  | ~90% glass windows shatter     |     3.5      |
|  3  | Wood frame buildings collapse  |     27      |
|  4  | Multistory brick buildings collapse  |     43      |

<p>
<div align="center">Table 1: Pressure thresholds (in kPa) for airblast damage</div>

According to the equations that we will use in this work, an asteoroid of approximately 7-m radius is required to generate overpressures on the ground exceeding 1 kPa, and an asteoroid of approximately 35-m radius is required to generate overpressures on the ground exceeding 43 kPa.

## Notes on distance, bearing and position

To determine the surface zero location (the point on Earth's surface that is closest to the point of airburst) a useful set of spherical geometric formulae relate the bearing, $\beta$ (also known as forward azimuth) to take to get from one point to another along a great circle,

$$\tan \beta = \frac {\cos \varphi_2\sin (\lambda_2-\lambda_1)}{\cos\varphi_1\sin\varphi_2-\sin\varphi_1\cos\varphi_2\cos(\lambda_2-\lambda_1)},$$

where $\lambda$ is longitude and $\varphi$ is latitude, as well as the related problem of the final destination given a surface distance and initial bearing:

$$\sin \varphi_2 = \sin \varphi_1\cos \left(\frac{r}{R_p}\right) +\cos \varphi_1\sin\left(\frac{r}{R_p}\right)\cos \beta,$$

$$ \tan(\lambda_2-\lambda_1) = \frac{\sin\beta\sin\left(\frac{r}{R_p}\right)\cos\varphi_1}{\cos\left(\frac{r}{R_p}\right)-\sin\varphi_1\sin\varphi_2}.$$

These formulae can all be derived from the spherical form of the [sine and cosine laws](https://en.wikipedia.org/wiki/Spherical_trigonometry#Cosine_rules_and_sine_rules) using relevant third points.

## Postcode locations

For those of you unfamiliar with UK postcodes, this [link](https://www.getthedata.com/postcode) might be helpful. Each postcode comprises of two strings of alpha-numeric characters that identify the geographic division of the UK. The first one or two letters of the first part of the postcode (before the number) identify the postcode **area** (e.g., WC); the whole of the first part of the postcode identifies the postcode **district**; the first part of the postcode, plus the first number of the second part of the postcode identifies the postcode **sector**. In this project, we will use the full postcode and the postcode sector.

<img src="images/postcode_map.png" width="640">

The geographic data supplied by running the `download_data.py` script consists of two files. The larger file is `full_postcodes.csv`, which contains a list of current UK postcodes, along with a government-assigned code designating the local administrative area and information on the average (mean) longitude and latitude of the addresses comprising the unit, using the international WGS 84 geodetic datum as supported by modern GPS.

In [1]:
import pandas as pd
postcodes = pd.read_csv('./resources/full_postcodes.csv')
postcodes.head()

,Postcode,LAU218CD,Latitude,Longitude
0,AB101AB,S31000932,57.149606,-2.096916
1,AB101AF,S31000932,57.148707,-2.097806
2,AB101AG,S31000932,57.149051,-2.097004
3,AB101AH,S31000932,57.148080,-2.094664
4,AB101AL,S31000932,57.150058,-2.095916


The smaller file is `population_by_postcode_sector.csv`, which contains 2011 census data arranged by postcode sector. The important columns for this project are the postcode sector ("geography") and the total population ("All usual residents"), although you are welcome to use other data in your tool if you wish.

In [2]:
census = pd.read_csv('./resources/population_by_postcode_sector.csv')
census.head()

,date,geography,geography code,Rural Urban,Variable: All usual residents; measures: Value,Variable: Males; measures: Value,Variable: Females; measures: Value,Variable: Lives in a household; measures: Value,Variable: Lives in a communal establishment; measures: Value,Variable: Schoolchild or full-time student aged 4 and over at their non term-time address; measures: Value,Variable: Area (Hectares); measures: Value,Variable: Density (number of persons per hectare); measures: Value
0,2011,AL1 1,AL1 1,Total,5453,2715,2738,5408,45,75,225.63,24.2
1,2011,AL1 2,AL1 2,Total,6523,3183,3340,6418,105,77,286.59,22.8
2,2011,AL1 3,AL1 3,Total,4179,2121,2058,4100,79,46,97.12,43.0
3,2011,AL1 4,AL1 4,Total,9799,4845,4954,9765,34,285,244.75,40.0
4,2011,AL1 5,AL1 5,Total,10226,5129,5097,10211,15,133,200.93,50.9


## Notes on longitude, latitude and distance

Given a pair of points by longitude and latitude, converting this into a distance between them can be a surprisingly involved calculation, involving a successively improving model of the shape of the Earth (the geoid). At the lowest reasonable level of approximation, in which the Earth is considered spherical, points at the same longitude satisfy a formula
$$|\varphi_1 -\varphi_2| = \frac{r}{R_p}$$
where the $\varphi$s are the latitudes (in radians), $r$ the surface distance between the points and $R_p$ the radius of the earth. As long as $r$ and $R_p$ are in the same units, the choice doesn't matter, but metres are usually to be preferred. For points at the same latitude, a similar formula applies, 
$$|\lambda_1 -\lambda_2| = \frac{r}{R_p\cos\varphi},$$
where the $\lambda$s are the longitudes and the $\varphi$ is the common latitude. In the general case a number of different formulas exist. [Among the more popular](https://en.wikipedia.org/wiki/Great-circle_distance) are the Haversine formula
$$\frac{r}{R_p} = 2\arcsin\sqrt{\sin^2 \frac{|\varphi_1-\varphi_2|}{2}+\cos\varphi_1\cos\varphi_2\sin^2\frac{|\lambda_1-\lambda_2|}{2}},$$
the spherical Vincenty formula
$$\frac{r}{R_p}=\arctan\frac{\sqrt{(\cos\varphi_2\sin|\lambda_1-\lambda_2|)^2+(\cos\varphi_1\sin\varphi_2-\sin\varphi_1\cos\varphi_2\cos|\lambda_1-\lambda_2|)^2}}{\sin\varphi_1 \sin\varphi_2+\cos\varphi_1\cos\varphi_2\cos|\lambda_1-\lambda_2|},$$
and the law of spherical cosines,
$$\frac{r}{R_p}=\arccos\left(\sin\varphi_1\sin\varphi_2+\cos\varphi_1\cos\varphi_2\cos|\lambda_1-\lambda_2|\right).$$
At short distances linearizations such as Pythagoras can also be used. 

Which formulae to choose is a balance between the cost of calculation and the accuracy of the result, which also depends on the specifics of the implementation. For example the two argument (also called `arctan2`) inverse tangent function should be preferred when needed (and available). In general the cheaper formulas have fewer trignometric function evaluations and square root calculations.

For this project, you should assume a spherical Earth and use one of the above approximations, but you may be interested to know that at the next level of approximation, the Earth is considered as an oblate spheriod (i.e. flattened sphere) and the full, iterative version of [Vincenty's formulae](https://en.wikipedia.org/wiki/Vincenty%27s_formulae) can be used. Further improvement includes local effects and acknowledges the implications of land elevation, but that sits well outside the scope of this exercise.

## Extended functionality

Additional credit will be given if your damage mapper function demonstrates the following extended capabilities:

* The ability to present the software output on a map. The graphics should be designed to be appropriate for use in emergency response and evacuation planning.
* The ability to perform a simple uncertainty analysis that takes as input a small uncertainty on each input parameter and calculates a risk for each affected UK postcode (sector).

### Plotting on a map

As one possible approach, we have provided a function to plot a circle on a map using the `folium` package. You can use `folium` and expand on this function or you may prefer to use a different package. Please check with us that the mapping package you wish to use is permissible before you start.

In [3]:
import folium
import armageddon
armageddon.plot_circle(53., 0, 2000.) #Plots a circle of radius 2000 m at the lat, lon: 53., 0.

### Uncertainty analysis

For this second extension exercise, a separate function `impact_risk` should be written that takes an additional set of inputs, describing the standard deviation of each input parameter, as well as the nominal input parameters. The uncertainty in each input parameter can be assumed to follow a gaussian distribution centered on the nominal values. The standard deviations for the parameters can be taken as:

* Entry latitude 0.025$^\circ$
* Entry longitude: 0.025$^\circ$
* Entry bearing: 0.5$^\circ$
* Meteoroid radius: 1 m
* Meteoroid speed: 1000 m/s
* Meteoroid density: 500 kg/m$^3$
* Meteoroid strength: 50\%
* Meteoroid trajectory angle: 1$^\circ$

For the second extension task, risk will be defined as the probability that the postcode sector (or postcode) is within a specified damage zone times the affected population. This function should therefore take as an input the overpressure used in the risk calculation and a flag to indicate whether risk should be calculated at the postcode or postcode sector level. For scoring, we will use damage level 3 (wooden buildings collapse) and postcode sectors.

Your risk calculator should sample the model parameter space $n$ times, where $n$ is an input parameter, but the sampling method is up to you. The probability that a postcode (or sector) is within a specified damage level is defined as the number of times the postcode (sector) is within the specified damage level divided by $n$. 

The risk calculator should output a Pandas dataframe with two columns: postcode (unit or sector) and risk.